In [1]:
import pandas as pd
import os, sys, glob
from consensusTarget import *
from consensusDecoy import *
from idtxtMs2ModsFunctions import *
from RTfunctions import *
import re
import collections #for ordering the dictionary
import pickle
import fileinput

import configparser
config = configparser.ConfigParser()


In [10]:
# # Read the parameter file using configparser

params_file = "specLib.params"

In [11]:
config.read(params_file)

['specLib.params']

In [12]:
#unimod modification information
unimod_modPklFile = config["specLib"]["unimod_modPklFile"]
 
#example pepxml file to parse the modification information
pepxml = config["specLib"]["pepxml"]
#results directory where mass correction results are present
resultsDirectory = config["specLib"]["resultsDirectory"]
#jump -f results ID.txt
idtxt = config["specLib"]["jump_f_dyn"]
#mzxml path that contains folder that have mzxml files
mzxml_path = config["specLib"]["mzxml_path"]

#Output folder name that contains spectral library files
specLibFolder = config["specLib"]["specLibFolder"]

#decoy generation distance
distanceDecoy = float(config["specLib"]["distanceDecoy"])

In [5]:
#get the unimod ptm dictionary unimod_mod_infor using the pickle file from unimod
with open(unimod_modPklFile, 'rb') as unimod_handle:
    unimod_mod_infor = pickle.load(unimod_handle)

In [6]:
jump_modAA_dict, jump_mod_dict, sta_AA = getDynStatModsInfoPepXml(pepxml)

In [13]:
ms2List = glob.glob(resultsDirectory+"/ms2/*.theo.ms2pep") #all theoretical ions for consensus
#idtxt = "sum_jumpf/ID.txt"  #this we need for advanced annotation of L in ms2 file after consensus
mzXMLs = glob.glob(mzxml_path+"/*/*.mzXML")


In [14]:
#Get RT information
#this is now changed based on Ji-Hoon's new program
mzRT_dict = getRT(idtxt,mzXMLs)

  Extraction and assignment of RTs to the identified PSMs
  Read ID.txt file: to extract PSM information
  Done ...
  RT of every identified peptide in FTLD_Batch2_F51 is being inferred and assigned
  Progress: [####################] 100% Done...
  RT of every identified peptide in FTLD_Batch2_F50 is being inferred and assigned
  Progress: [####################] 100% Done...
  RT of every identified peptide in FTLD_Batch2_F52 is being inferred and assigned
  Progress: [####################] 100% Done...

  Alignment and calibration of RTs over multiple runs
  FTLD_Batch2_F51 is set to the initial reference for RT-alignment
  The RTs in FTLD_Batch2_F50 are being aligned and calibrated
  The RTs in FTLD_Batch2_F52 are being aligned and calibrated
  Done ...



In [15]:
psmsDict,precDict = ms2fileToDict(ms2List)

idtxtConsensus2 = cosensusPSMTable(idtxt, psmsDict, precDict, jump_modAA_dict, jump_mod_dict, sta_AA, unimod_mod_infor,mzRT_dict)

idtxtConsensus3 = psmConsolidate(idtxtConsensus2)

  All ms2 are now stored as dictionary
  Working with  /Users/spoudel1/Desktop/JUMP_specLib/Program/finalProgram/JUMPp-lib/MassCalResults/ms2/FTLD_Batch2_F50.theo.ms2pep 

  Done for ...  /Users/spoudel1/Desktop/JUMP_specLib/Program/finalProgram/JUMPp-lib/MassCalResults/ms2/FTLD_Batch2_F50.theo.ms2pep 

  Working with  /Users/spoudel1/Desktop/JUMP_specLib/Program/finalProgram/JUMPp-lib/MassCalResults/ms2/FTLD_Batch2_F51.theo.ms2pep 

  Done for ...  /Users/spoudel1/Desktop/JUMP_specLib/Program/finalProgram/JUMPp-lib/MassCalResults/ms2/FTLD_Batch2_F51.theo.ms2pep 

  Working with  /Users/spoudel1/Desktop/JUMP_specLib/Program/finalProgram/JUMPp-lib/MassCalResults/ms2/FTLD_Batch2_F52.theo.ms2pep 

  Done for ...  /Users/spoudel1/Desktop/JUMP_specLib/Program/finalProgram/JUMPp-lib/MassCalResults/ms2/FTLD_Batch2_F52.theo.ms2pep 

  Constructing a consensus table for unique psms

Done ...

  psms consolidation begin
  Top 10 psms from all fractions are now consolidated



In [16]:
#make directory spectral library folder
cmd1 = "mkdir "+specLibFolder
try:
  os.system(cmd1)
except:
  pass

In [17]:
targetDF = createMS2EachPSMS_L_ID(idtxtConsensus3, specLibFolder)

decoySpecLibrary(targetDF, specLibFolder, distanceDecoy)


  Generating .ms2 files

  now = 2021-04-27 21:52:09.186791
  Target IDs spectral library is being created /Users/spoudel1/Desktop/JUMP_specLib/Program/finalProgram/specLib/Spectral_Library_theo_mz_exp_intensity.spLib
  Done ...

  Generating .ms2 files

  now = 2021-04-27 21:52:10.952889
  Decoy spectral library is being created
  Done ...



In [18]:
specLib = specLibFolder+"/Spectral_Library_theo_mz_exp_intensity.spLib"
decoyLib = specLibFolder+"/Spectral_Library_Decoy.spLib"
outfilename = specLibFolder+"/SpectralLibraryTargetDecoy.spLib"


In [19]:
filenames = [specLib,decoyLib]

In [20]:
#concatenate two spectral library
with open(outfilename, 'w') as fout, fileinput.input(filenames) as fin:
    for line in fin:
        fout.write(line)
